In [61]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.animation as anm
%matplotlib widget
import matplotlib.pyplot as plt

class World(object):
    def __init__(self, debug=False):
        self.objects = []
        self.debug = debug
    
    def append(self, obj):
        self.objects.append(obj)
    
    def draw(self):
        fig = plt.figure(figsize=(4,4))
        ax = fig.add_subplot(111)
        ax.set_aspect('equal')
        ax.set_xlim(-5, 5)
        ax.set_ylim(-5, 5)
        ax.set_xlabel('X', fontsize=20)
        ax.set_ylabel('Y', fontsize=20)
        
        elems = []
        
        if self.debug:
            for i in range(1000):
                self.one_step(i, elems, ax)
        else:
            self.ani = anm.FuncAnimation(fig, self.one_step, fargs=(elems, ax), frames=50, interval=1000, repeat=False)
            plt.show()

    def one_step(self, i, elems, ax):
        while elems:
            elems.pop().remove()
        elems.append(ax.text(-4.4, 4.5, "t = "+str(i), fontsize=10))
        for obj in self.objects:
            obj.draw(ax, elems)
            if hasattr(obj, "one_step"):
                obj.one_step(1.0)

In [40]:
world = World()

In [41]:
world.draw()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
import numpy as np
import math
import matplotlib.patches as patches

In [54]:
class IdealRobot(object):
    def __init__(self, pose, agent=None, color='black'):
        self.pose = pose
        self.r = 0.2
        self.agent = agent
        self.color = color
        self.poses = [pose]
    
    def draw(self, ax, elems):
        x, y, theta = self.pose
        xn = x + self.r*math.cos(theta)
        yn = y + self.r*math.sin(theta)
        elems += ax.plot([x, xn], [y, yn], color=self.color)
        c = patches.Circle(xy = (x,y), radius=self.r, fill=False, color=self.color)
        elems.append(ax.add_patch(c))
        
        self.poses.append(self.pose)
        elems += ax.plot([e[0] for e in self.poses], [e[1] for e in self.poses], linewidth=0.5, color='black')
    
    def one_step(self, time_interval):
        if not self.agent:
            return
        nu, omega = self.agent.decision()
        self.pose = self.state_transition(nu, omega, time_interval, self.pose)
    
    @classmethod
    def state_transition(cls, nu, omega, dt, pose):
        theta = pose[2]
        if math.fabs(omega) < 1e-5:
            return pose + np.array([nu*math.cos(theta), nu*math.sin(theta), omega]) * dt
        else:
            return pose + np.array([nu/omega*(math.sin(theta+omega*dt)-math.sin(theta)), nu/omega*(-math.cos(theta+omega*dt)+math.cos(theta)), omega*dt])

In [49]:
IdealRobot.state_transition(0.1, 0.0, 1.0, np.array([0,0,0]).T)
IdealRobot.state_transition(0.1, 10.0/180.0*math.pi, 9.0, np.array([0,0,0]).T)

array([0.5729578 , 0.5729578 , 1.57079633])

In [51]:
class Agent(object):
    def __init__(self, nu, omega):
        self.nu = nu
        self.omega = omega
    
    def decision(self, observation=None):
        return self.nu, self.omega

In [62]:
world = World()
lin = Agent(0.2, 0.0)
ver = Agent(0.2, 10.0/180.0*math.pi)
robot1 = IdealRobot(np.array([2,3,math.pi/6]).T, lin, 'green')
robot2 = IdealRobot(np.array([-2,-1,math.pi/5*6]).T, ver, 'red')
robot3 = IdealRobot(np.array([0,0,0]).T, color='blue')
world.append(robot1)
world.append(robot2)
world.append(robot3)
world.draw()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …